<a href="https://colab.research.google.com/github/hongsy0113/cose461-NLP-project-choi-hong/blob/main/Naive_bayes_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import json
from pandas import json_normalize


In [ ]:
# json data 파일들을 google drive에 저장
# google drive 에 있는 파일들을 접근하기 위해 mount
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [58]:
# Naive-Bayes Method.
df = pd.read_csv('/content/gdrive/MyDrive/cose461/data_100.csv')
df.head(10)

,index,utterance,P_gender
0,"('9fdbaeb2-3f1a-5ba2-a6a6-53d9a57fd33d', 'P02')",학교 6일에감 내일가냐? 불썅 ㅇㅎㅇㅎ ㅇㅇㅇㅇ왜안산에있어 다1교시얔ㅋㅋ,0
1,"('3fb66713-37a2-50e5-9b0a-afb898c06eea', 'P02')","천처니 오십셔,, 한 다섯시 맞춰서 오면 되지않으까 싶네욤 ㅎㅎ 흑흑 보고파 오뤼 ...",0
2,"('642fd586-be49-5bd9-b596-ccb2c8badab6', 'P02')",ㅋㅋㅋㅋㅋㅋㅋ 더워? 하긴여기도 그렇게안추워 ㅋㅋㅋㅋㅋㅋ벌써마지막이야?? 오늘 하루...,0
3,"('b8d86fc3-2154-562c-b8d6-bb566d72fa4f', 'P02')",딱히읍지 나야 왜?? 인스타보니까 놀러가신거? 13일에 오는거야? 구래 볼수있늠 보...,0
4,"('ce82e17c-5730-5ed4-8bf9-40eac1f5d99a', 'P01')","속초에 월, 화 비온다고 되어있는데 화부턴 날씨가 추워지더라고 2도 내생각엔 오빠 ...",0
5,"('c4cda8af-f443-5add-9c5d-c17859c975ae', 'P01')",근데 커피를 저기다가 흘릴일이없거든 ... ㅋㅋㅋㅋ 배고프당 너무너무너무너무 나 지...,0
6,"('a700867e-3173-5b98-b6fb-f8648d6d7fea', 'P02')",아니~ 단식원은 가본적 없어 단식원을 왜 가~ 가서 고생만 해 어차피 그런거는 다시...,0
7,"('d40c6bef-e632-5a8c-b2ac-2e910f8b3bf6', 'P02')","제관 팀플장소 차서 카퍼ㅣ드림 왔어 아이쿠,, 글게말야 ㅋㅋㅋㅋㅋㄱ 있오 나 수업 ...",0
8,"('93ba610a-57ed-5d0e-991e-1698ebeebb51', 'P02')",주말 평일은 시간없는뎅 토욜밖에안됨 ㅇㅇ 나도 그때 제본이랑 짐정리해야됨,0
9,"('9bd9dfd5-e432-5d15-aaff-17a3c2cdf440', 'P01')",한번 배워보고싶긴하다 오빠한테 물어봐야겠네 아~ 카페에서 커피 여유롭게 마시고싶다 ...,0


In [ ]:
from soynlp.normalizer import *
from konlpy.tag import Okt
import re

okt = Okt()

def clean(doc) :

  new_doc = list()
  doc = okt.pos(doc, norm=True)

  stop_tags = ['Determiner', 'Josa', 'Foreign']
  stop_words = ['은', '는', '이', '가', '', '이름', '계정', '주소', '신원', '전번', '금융', '번호', '소속', '기타']
  for text, tag in doc:  

    if tag in stop_tags:
      continue

    text = re.sub(r'[^ㄱ-ㅣ가-힣?.!~\^]+', '', text)  # remove digits.  
    text = emoticon_normalize(text, num_repeats=2) # remove repeated emoticon. e.g) ㅋㅋㅋㅋ=>ㅋㅋ, ㅠㅠㅠㅠ=>ㅠㅠ
    text = repeat_normalize(text, num_repeats=1) # remove repeated character
    
    if text in stop_words or (tag=='Verb' and len(text)<=1):
      continue
      
    new_doc.append(text)

  return new_doc


In [ ]:
df['sents'] = df['utterance'].apply(clean)

In [ ]:
X = df['sents']
y = df['P_gender']

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [ ]:
vectorizer = CountVectorizer().fit(X_train)
X_train_countvector = vectorizer.transform(X_train)

feature_names = vectorizer.get_feature_names()
len(feature_names)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


593197

In [ ]:
from sklearn.model_selection import cross_validate

classifier = MultinomialNB()
scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

cv_results = cross_validate(classifier, X_train_countvector, y_train, scoring=scoring, cv=5)

In [ ]:
# cv_results.keys()
print(np.mean(cv_results['test_accuracy']))
print(np.mean(cv_results['test_precision'])) 
print(np.mean(cv_results['test_recall']))
print(np.mean(cv_results['test_f1']))
print(np.mean(cv_results['test_roc_auc']))


0.6841506838137661
0.6976314137575063
0.6503067733704373
0.6731378429263541
0.7567343460963647


In [ ]:
# Final test.

classifier = MultinomialNB()
classifier.fit(X_train_countvector, y_train)

y_predict = classifier.predict(vectorizer.transform(X_test))
cm = confusion_matrix(y_test, y_predict)
df_cm = pd.DataFrame(cm, index = ['실제값(0)', '실제값(1)'], columns = ['예측값(0)', '예측값(1)'])
print(classification_report(y_test, y_predict))


              precision    recall  f1-score   support

           0       0.68      0.72      0.70    100143
           1       0.70      0.65      0.68     99955

    accuracy                           0.69    200098
   macro avg       0.69      0.69      0.69    200098
weighted avg       0.69      0.69      0.69    200098



In [ ]:
print(df_cm)

        예측값(0)  예측값(1)
실제값(0)   72360   27783
실제값(1)   34776   65179
